# Query Reward Unit Test
Performing verification of the query rewards mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

stakeDelegateds: 605 events
stakeDelegatedLockeds: 375 events
stakeDelegatedWithdrawns: 326 events
allocationCloseds: 47 events
allocationCollecteds: 16 events
stakeDepositeds: 25 events
rewardsAssigneds: 47 events
delegationParametersUpdateds: 6 events
TOTAL NUMBER OF EVENTS: 1529
Set SIMULATION_TIME_STEPS in config.py to 1529



Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
# print(query_fee_events)
query_fee_events_dict = query_fee_events
print("EXPECTED TRUE QUERY REWARDS FROM STAKE DEPOSITED EVENTS:")
for timestep, query_fee_events_list in query_fee_events_dict.items():
    if query_fee_events_list is None:
        query_fee_amt = 0
    else:
        query_fee_amt = sum([e['tokens'] for e in query_fee_events_list])
    print(f"{timestep}, {query_fee_amt}")
    


EXPECTED TRUE QUERY REWARDS FROM STAKE DEPOSITED EVENTS:
465, 137.509689357001388186
703, 26.735411802895408127
704, 177.886072492066675018
778, 214.529777397159839053
818, 18.276136055873106202
819, 18.276136055873106202
820, 18.276136055873106202
821, 18.276136055873106202
822, 18.276136055873106202
823, 18.276136055873106202
824, 18.276136055873106202
825, 18.276136055873106202
826, 18.276136055873106202
1079, 7.259243672384625343
1090, 1.200768983572724847
1105, 1.200768983572724847


In [4]:
print("MODELED RESULTS")
for timestep, query_fee_events_list in query_fee_events_dict.items():    
    query_fee_amt = df.iloc[timestep-1]['cumulative_query_revenue'] - df.iloc[timestep-2]['cumulative_query_revenue']
    print(f"{timestep}, {query_fee_amt=}")


MODELED RESULTS
465, query_fee_amt=Decimal('137.509689357001388186')
703, query_fee_amt=Decimal('26.735411802895408127')
704, query_fee_amt=Decimal('177.886072492066675018')
778, query_fee_amt=Decimal('214.529777397159839053')
818, query_fee_amt=Decimal('18.276136055873106202')
819, query_fee_amt=Decimal('18.276136055873106202')
820, query_fee_amt=Decimal('18.276136055873106202')
821, query_fee_amt=Decimal('18.276136055873106202')
822, query_fee_amt=Decimal('18.276136055873106202')
823, query_fee_amt=Decimal('18.276136055873106202')
824, query_fee_amt=Decimal('18.276136055873106202')
825, query_fee_amt=Decimal('18.276136055873106202')
826, query_fee_amt=Decimal('18.276136055873106202')
1079, query_fee_amt=Decimal('7.259243672384625343')
1090, query_fee_amt=Decimal('1.200768983572724847')
1105, query_fee_amt=Decimal('1.200768983572724847')


In [5]:
print("UNITTEST RESULTS")
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, query_fee_events_list in query_fee_events_dict.items():
    if query_fee_events_list is None:
        event_query_fee_amt = 0
    else:
        event_query_fee_amt = sum([e['tokens'] for e in query_fee_events_list])
    model_query_fee_amt = df.iloc[timestep-1]['cumulative_query_revenue'] - df.iloc[timestep-2]['cumulative_query_revenue']
    print(f"Event: Timestep={timestep-1}, Query Reward Tokens={event_query_fee_amt}")
    print(f"Model: Timestep={timestep-1}, Query Reward Tokens={model_query_fee_amt}")
    # print("Query Rewards Equal?", event_query_fee_amt == model_query_fee_amt)
    # print()

    try:
        ratio =  abs(event_query_fee_amt / model_query_fee_amt)
    except:
        ratio = np.inf
    exact = ratio == 1.0
    close = 0.99 <= ratio <= 1.01
    reallyClose = 0.99999 <= ratio <= 1.00001

    if exact:            
        cntExact += 1
    elif reallyClose:
        cntReallyClose += 1
    elif close:
        cntClose += 1
    else:
        cntWrong += 1
    cnt += 1
    print("Tokens Equal?", exact)
    print("Tokens Within 0.001%?", reallyClose)
    print("Tokens Within 1%?", close)
    print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    
    


UNITTEST RESULTS
Event: Timestep=464, Query Reward Tokens=137.509689357001388186
Model: Timestep=464, Query Reward Tokens=137.509689357001388186
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 1, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 1

Event: Timestep=702, Query Reward Tokens=26.735411802895408127
Model: Timestep=702, Query Reward Tokens=26.735411802895408127
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 2, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 2

Event: Timestep=703, Query Reward Tokens=177.886072492066675018
Model: Timestep=703, Query Reward Tokens=177.886072492066675018
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 3, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 3

Event: Timestep=777, Query Reward Tokens=214.529777397159839053
Model: Timestep=777, Query Reward Tokens=214.529777397159839053
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 4

# Conclusion
The query reward token quantities are exactly equal.